<a href="https://colab.research.google.com/github/shashwattrivedi/Simple-TransformerEncoder-keras/blob/master/Transformertest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [2]:
!git clone https://github.com/shashwattrivedi/Simple-TransformerEncoder-keras.git
!git clone https://github.com/shashwattrivedi/Attention_visualizer.git

Cloning into 'Simple-TransformerEncoder-keras'...
remote: Enumerating objects: 110, done.
remote: Counting objects: 100% (110/110), done.
remote: Compressing objects: 100% (110/110), done.
remote: Total 110 (delta 62), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (110/110), 591.06 KiB | 3.16 MiB/s, done.
Resolving deltas: 100% (62/62), done.
Cloning into 'Attention_visualizer'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 44 (delta 10), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (44/44), done.


In [0]:
!cp Simple-TransformerEncoder-keras/Transformer/* .
!cp Simple-TransformerEncoder-keras/Polarity_model/* .
!cp Attention_visualizer/attention_visualizer.py .

In [4]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import Tokenizer
import keras.backend as K
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import spacy

Using TensorFlow backend.


In [0]:
from TransformerEncoder import TransformerEncoder
from TrainablePositionalEmbeddings import TransformerPositionalEmbedding
from utility import *
from keras.layers import Input,Dense,Embedding,Add,Lambda,Flatten,Reshape
from keras.models import Model
import pandas as pd
import numpy as np
from preprocess import *
from attention_visualizer import *

In [0]:
# data directory
glovePath = '/content/gdrive/My Drive/Colab Notebooks/glove.6B.50d.txt'
dataPath  = 'polarity.pkl'

#**Hyper-parameters**

In [0]:
# Hyper-parameters
word_embeddings_dim = 50
heads               = 2
query_dim           = 32
value_dim           = 32
positional_ff_dim   = 128




# Reading data and Preprocess

In [0]:
#Reading
data,labels,label_names,val,val_labels = load_data_from_pickle(dataPath)

#Preprocess
nlp           = spacy.load('en')
textprocessor = TextProcessor(nlp)
data          = textprocessor.fit_and_transform(data)
val           = textprocessor.transform(val)

#Train-test Split
train_X, test_X, train_y, test_y = data, val, labels, val_labels

# Preprocess input

In [9]:
glove_embeddings  = loadGlove(glovePath)
emb_mat           = getEmbMatrix(glove_embeddings,textprocessor.tokenizer.word_index,word_embeddings_dim)

(17682, 50)


# Model

In [0]:
def getPolarityModel(max_sequence_len,embedding_matrix,word_embeddings_dim,
                     transformer_depth,heads,
                     query_dim,value_dim,positional_ff_dim):
    
    X                             = Input(shape=(max_sequence_len,),dtype='int32')
    #mask_layer                    = Reshape((max_sequence_len,1))(MaskLayer()(X)) #for masking
    wordEmb                       = Embedding(len(emb_mat), 
                                              word_embeddings_dim, 
                                              weights=[emb_mat], 
                                              trainable=False,
                                              name='Word_Embedding')
    positional_embedding_layer    = TransformerPositionalEmbedding(name='Positional_embedding')
    
    next_step_input               = positional_embedding_layer(wordEmb(X))
    #next_step_input               = Lambda(lambda x: x[0]*x[1])([next_step_input, mask_layer]) 

    attention=[None for i in range(transformer_depth)]
    for i in range(transformer_depth):
            next_step_input,attention[i] =TransformerEncoder(word_embeddings_dim,
                                                             heads,
                                                             query_dim,
                                                             value_dim,
                                                             positional_ff_dim,
                                                             dropout_rate= 0.1,
                                                             name= 'Transformer'+str(i))(next_step_input) 
            
    
    sentence_representation       = Lambda(lambda x: x[:,0,:])(next_step_input)
    review_class                  = Dense(1,activation='sigmoid',name='Polarity')(sentence_representation)

    model                         = Model(inputs=[X],outputs=[review_class,attention[-1]])
    
    return model
    
    
    

In [0]:
classifier = getPolarityModel(textprocessor.max_seq_len,
                              emb_mat,word_embeddings_dim,
                              transformer_depth=1,
                              heads = 4,
                              query_dim = 128,
                              value_dim = 128,
                              positional_ff_dim=256)

In [12]:
classifier.compile('nadam',
                  loss={'Polarity': 'binary_crossentropy'},
                  metrics={'Polarity': 'accuracy'}                
                  )


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Output "lambda_6" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "lambda_6" during training.
  This is separate from the ipykernel package so we can avoid doing imports until


In [17]:
classifier.fit([train_X],[train_y],batch_size=32,epochs=5,shuffle=True)

Epoch 1/5
10433/10433 [==============================] - 5s 511us/step - loss: 0.5188 - Polarity_loss: 0.5188 - Polarity_acc: 0.7415
Epoch 2/5
10433/10433 [==============================] - 5s 498us/step - loss: 0.4972 - Polarity_loss: 0.4972 - Polarity_acc: 0.7538
Epoch 3/5
10433/10433 [==============================] - 5s 498us/step - loss: 0.4898 - Polarity_loss: 0.4898 - Polarity_acc: 0.7633
Epoch 4/5
10433/10433 [==============================] - 5s 496us/step - loss: 0.4890 - Polarity_loss: 0.4890 - Polarity_acc: 0.7609
Epoch 5/5
10433/10433 [==============================] - 5s 497us/step - loss: 0.4818 - Polarity_loss: 0.4818 - Polarity_acc: 0.7657


In [18]:
classifier.evaluate([test_X],test_y)

1000/1000 [==============================] - 0s 148us/step


[0.46649920296669006, 0.46649920296669006, 0.765]

In [0]:
def try_this_sentence(sentence,head):
  inp = textprocessor.transform([sentence])
  y_pred, attention = classifier.predict(inp)
  print("The prediction value is " + str (y_pred[0][0]) + " That is " +str(label_names[y_pred[0][0]>=0.5]))
  
  input_sent = textprocessor.backtransform(inp)[0].split()
  return display_attention(input_sent+["" for _ in range(len(attention[0,head-1,0,:])-len(input_sent))],list(map(float,attention[0,head-1,0,:])))
  

In [20]:
try_this_sentence('I liked the movie, it was entertaining',3)

The prediction value is 0.77394795 That is positive


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  after removing the cwd from sys.path.


In [22]:
try_this_sentence('I liked the movie, it was entertaining',4)

The prediction value is 0.77394795 That is positive


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  after removing the cwd from sys.path.
